## BCH/Repetition Product Code

In [1]:
class BCHREPCode:
    
    def __init__(self, n_BCH, b, D, n_REP):
        
        self.q = 2
        self.n_BCH = n_BCH
        self.m = Mod(self.q,self.n_BCH).multiplicative_order()
        self.b = b
        self.D = D
        self.n_REP = n_REP
            
        #if (gcd(self.n_BCH, self.q) != 1):
        #    raise ValueError('Invalid input values: gcd(n,q) != 1.')
            
        if (self.n_BCH != self.q**self.m - 1):
            raise ValueError('Invalid input values: n != q^m - 1')
            
        
        # Initialize field
        self.BF = GF(self.q) # base field
        self.EF = GF(self.q**self.m) # extension field
        R.<x> = PolynomialRing(self.BF, 'x')
        self.R = R
        self.x = x
        self.R_RS = PolynomialRing(self.EF, 'X')
        self.alpha = self.EF.primitive_element()
        
        # define RS parameters
        self.k_RS = self.n_BCH - self.D + 1
        self.tau_RS = floor( (self.D - 1) / 2)
        self.alpha_RS = vector([self.alpha**i for i in range(self.n_BCH)])
        self.G_RS = matrix(self.EF, self.k_RS, self.n_BCH, lambda i,j : self.alpha_RS[j]**i)
        
        # Constructing generator matrix
        self.cosets = self.cyclotomic_cosets(self.n_BCH, self.q, self.b, self.D)
        
        self.generator_poly = self.BCH_generator_polynomial(self.x, self.alpha, self.D, self.cosets)
        
        if not (self.generator_poly.divides(self.x**self.n_BCH - 1)):
            raise ValueError('generator_poly is not a generator polynomial')
            
        self.k = self.n_BCH - self.generator_poly.degree()
        
        self.G_BCH = matrix(self.BF, self.k, self.n_BCH, lambda i,j : self.generator_poly[(j+(self.n_BCH-i)) % self.n_BCH])
        
        # Parameters for product code
        self.code_n = self.n_BCH * self.n_REP
        self.code_k = self.k
        self.code_d = self.D * self.n_REP
        self.code_delta = floor((self.code_d - 1) / 2)
        
        
    def cyclotomic_cosets(self, n, q, b, D):
        # compute cyclotomic cosets
    
        cosets = []

        for i in range(b,b+D-1):
            coset = [(i * q**j) % n for j in range(0,n-1)]
            coset = list(set(coset))
            coset.sort()
            cosets.append(coset)
        return cosets

    def minimal_polynomial(self, coset, x, alpha):
        # compute minimal polynomial from one coset
        poly = 1
        for j in range(len(coset)):
            poly *= (x - alpha**coset[j])
        return poly

    def BCH_generator_polynomial(self, x, alpha, D, cosets):
        # compute generator polynomial
        poly = self.minimal_polynomial(cosets[0], x, alpha)
        for i in range(1,D-1):
            poly = LCM(poly,self.minimal_polynomial(cosets[i],x,alpha))
        
        return poly
    
    
    def Encoding(self, message, zeropad = True):
        
        rem = len(message) % self.k
        
        if rem != 0:
            if zeropad:
                message.extend([self.BF(0)]*(self.k-rem))
            else:
                raise ValueError('k does not divide input size')
                
                
        c = []
        
        # Encoding each chunk of size k
        for i in range(0, len(message), self.k):
            c.extend(self.EncodeChunkBCHREP(message[i:i+self.k]))
        
        return c
            
    def EncodeChunkBCHREP(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k:
            raise ValueError('Invalid chunk size')
            
        # BCH encoder
        c_BCH = vector(self.BF, chunk) * self.G_BCH
        
        # Repetition encoder
        c_BCH_REP = []
        
        for i in range(len(c_BCH)):
            for j in range(self.n_REP):
                c_BCH_REP.append(c_BCH[i])
                
        return c_BCH_REP
                
    
    def Decoding(self, c_BCH_REP):
        
        # Check input size
        if len(c_BCH_REP) % (self.code_n) != 0:
            raise ValueError('Invalid input size')
            
        c_BCH = []
        
        # Decode with REP decoder
        for i in range(0,len(c_BCH_REP),self.n_REP):
            c_BCH.append(self.DecodeChunkREP(c_BCH_REP[i:i+self.n_REP]))
        
        # Need to convert to extension field
        for i in range(len(c_BCH)):
            c_BCH[i] = self.EF(c_BCH[i])
          
        # Decode with RS decoder
        c = []
        
        for i in range(0,len(c_BCH),self.n_BCH):
            c.extend(self.DecodeChunkRS(c_BCH[i:i+self.n_BCH]))
            
        # Encode with RS encoder
        c_RS = [] 
        
        for i in range(0, len(c), self.k_RS):
            c_RS.extend(self.EncodeChunkRS(c[i:i+self.k_RS]))
        
        # Decode with BCH decoder
        if len(c_RS) % self.n_BCH != 0:
            raise ValueError('Invalid codeword size')
            
        # Need to convert back to base field
        for i in range(len(c_RS)):
            c_RS[i] = self.BF(c_RS[i])
            
        c = []
        
        cols = self.G_BCH.pivots()
        G_independent = self.G_BCH.matrix_from_columns(cols)
        
        for i in range(0,len(c_RS),self.n_BCH):
            c.extend(self.DecodeChunkBCH(c_RS[i:i+self.n_BCH], cols, G_independent))
            
        return c
    
    def DecodeChunkBCH(self, chunk, cols, G_independent):
        
        chunk_independent = [chunk[i] for i in cols]
        
        return vector(self.BF, chunk_independent) * G_independent.inverse()
    
    
    def DecodeChunkREP(self, chunk):
        if chunk.count(1) >= ceil(self.n_REP / 2):
            return 1
        else:
            return 0
    
            
    def EncodeChunkRS(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k_RS:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.EF, chunk) * self.G_RS
        return c
    
    def DecodeChunkRS(self, chunk):
        # Bivariate interpolation is used
        
        if len(chunk) != self.n_BCH:
            raise ValueError('Invalid chunk size')
            
        # Constructing matrices
        M1 = matrix(self.EF, self.n_BCH, self.tau_RS + self.k_RS, lambda i,j : self.alpha_RS[i]**j)
        M2 = matrix(self.EF, self.n_BCH, self.tau_RS + 1, lambda i,j : chunk[i] * self.alpha_RS[i]**j)
        M = M1.augment(M2)
        
        # Solving system
        RK = M.right_kernel()
        
        if len(RK.basis()) == 0:
            return(None)
        
        sol = RK.basis()[0]

        # Constructing Q0 and Q1 polynomials
        Q0 = self.R_RS(list(sol[:self.tau_RS+self.k_RS]))
        Q1 = self.R_RS(list(sol[self.tau_RS+self.k_RS:]))

        # Calculating -Q0/Q1
        q, r = Q0.quo_rem(Q1)

        if r != 0:
            print('Non-zero remainder (possibly >tau errors). Returning None')
            return(None)

        out = []

        out.extend((-q).list())
        out.extend([self.EF(0)]*(self.k_RS-len(out)))

        return out

It is now shown that the product code can correct $\delta=\lfloor\frac{d_{BCH}d_{REP}-1}{2}\rfloor$ errors:

In [3]:
C = BCHREPCode(n_BCH = 1023, b = 1, D = 5, n_REP = 31)

m = []
for i in range(C.code_k):
    m.append(C.BF.random_element())
    

print('delta = ', C.code_delta)
    
c = C.Encoding(m)

positions = []
for i in range(C.code_delta): # add delta errors
    position = ZZ.random_element(0,C.code_n)
    while position in positions:
        position = ZZ.random_element(0,C.code_n)  
      
    positions.append(position)
    c[position] = C.BF(c[position] - 1) # flip the bit

d = C.Decoding(c)

print('Decoding status: ', d == m)

delta =  77
Decoding status:  True
